# Collect all settled locations

In [1]:
import geopandas as gpd
import pandas as pd
import geopy
import osmnx as ox
iq_locations = gpd.read_file('data/settled_locations/irq_pplp_ocha_20140722.shp')
location_csv = pd.DataFrame()
location_csv['name'] = iq_locations['PNameEn']
location_csv['region'] = iq_locations['A1NameEn']
location_csv['country'] = 'Iraq'
location_csv['lat'] = iq_locations['Latitude']
location_csv['lon'] = iq_locations['Longitude']
location_csv['location_type'] = 'town'
location_csv['conflict_date'] = 0
location_csv['pop/cap'] = 0

# Get population numbers

In [2]:
pop87 = pd.read_csv('data/undata_1987_census/UNdata_Export_20180505_224541956.csv')
pop87 = pop87[:-2] # drop nans
pop87

,Country or Area,Year,Area,Sex,City,City type,Record Type,Reliability,Source Year,Value,Value Footnotes
0,Iraq,1987,Total,Both Sexes,Adhamiyah,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,464151.0,NaN
1,Iraq,1987,Total,Both Sexes,Amara,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,208797.0,NaN
2,Iraq,1987,Total,Both Sexes,BAGHDAD,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,3841268.0,1.0
3,Iraq,1987,Total,Both Sexes,Basra,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,406296.0,NaN
4,Iraq,1987,Total,Both Sexes,Diwaniya,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,196519.0,NaN
5,Iraq,1987,Total,Both Sexes,Erbil,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,485968.0,NaN
6,Iraq,1987,Total,Both Sexes,Hilla,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,268834.0,NaN
7,Iraq,1987,Total,Both Sexes,Kadhimain,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,521444.0,NaN
8,Iraq,1987,Total,Both Sexes,Karradah Sharqiyah,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,235554.0,NaN
9,Iraq,1987,Total,Both Sexes,Kerbala,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,296705.0,NaN


In [97]:
replace_dict

{'Adhamiya': 0    464151.0
 Name: Value, dtype: float64}

In [163]:
iq_pop_bydist = gpd.read_file('data/Iraq_Districts/iraq_districts.shp')
iq_pop_bydist['num_settlements'] = 0
iq_pop_bydist['pop87'] = 0
iq_pop_bydist['pop_2015'] = 0
iq_pop_bydist['pred_pop'] = 0

replace_dict = {}
# this had to be done by hand to check spellings and varied Eng-Arabic transliteration
replace_dict['Adhamiya'] = pop87.Value[pop87.City== 'Adhamiyah']
replace_dict['Amara'] = pop87.Value[pop87.City== 'Amara']
# districts not given unique counts in 87: 
replace_dict['Tarmia'] = pop87.Value[pop87.City== 'Baghdad'] / 
replace_dict['Khadamiya'] = pop87.Value[pop87.City== 'Kadhimain']
replace_dict[''] = pop87.Value[pop87.City== 'Karradah Sharqiyah']

SyntaxError: invalid syntax (<ipython-input-163-04a59332f1da>, line 12)

# Above was taking too long, so just going to use worldpop estimates,  crs matches for locations and worldpop raster

In [ ]:
import rasterio as rio
pops = rio.open('data/IRQ_ppp_v2b_2015_UNadj/IRQ_ppp_v2b_2015_UNadj.tif')

import matplotlib.pyplot as plt
import numpy as np

In [157]:
pops_dat = pops.read().squeeze()
type(pops_dat)
#plt.imshow(pops_dat, cmap = 'hot')

numpy.ndarray

# now getting the routes between all locations
start with just doing great circle distance, then maybe move to routing values

In [15]:
dist_iq_locations = iq_locations.copy().sample(200)
dist_iq_locations.head()

,PCode,PNameEn,PNameAr,Longitude,Latitude,PCodeAlt1,PNameAlt1,A1Code,A1NameEn,A2Code,A2NameEn,geometry
14002,IQ-P13967,Bani Maran,,44.370000,36.040000,1106013.0,Bani Maran,IQ-G11,Erbil,IQ-D065,Koisnjaq,POINT (44.37 36.04)
12079,IQ-P12047,Muhammad Chawsh,,45.400000,33.700000,1005482.0,Muhammad Chawsh,IQ-G10,Diyala,IQ-D057,Baladrooz,POINT (45.4 33.7)
7334,IQ-P07317,Hudaymah,,44.630000,32.250000,603420.0,Hudaymah,IQ-G06,Babylon,IQ-D034,Hashimiya,POINT (44.63 32.25)
9380,IQ-P10059,Karuka,,43.770000,37.110000,804587.0,Karuka,IQ-G08,Dahuk,IQ-D048,Amedi,POINT (43.77 37.11)
21050,IQ-P21012,Qaryat Misqlat Alia,,43.042892,36.594774,1510950.0,Qaryat Misqlat Alia,IQ-G15,Ninewa,IQ-D091,Tilkaif,POINT (43.042892 36.594774)


In [248]:
dist_df = pd.DataFrame()
for i in dist_iq_locations.itertuples():
    dist_df[i.PNameEn] = dist_iq_locations.geometry.distance(i.geometry)
dist_df.index = dist_df.columns
dist_df.to_csv('distance_matrix.csv')

KeyboardInterrupt: 

In [34]:
trips = set()
count = 0
with open("data/adj_mat_test.csv", "w") as f:
    f.write(",".join(["name1", "name2", "distance\n"]))
    for i in dist_iq_locations.itertuples():
        dists = dist_iq_locations.geometry.distance(i.geometry)
        tups = zip(dist_iq_locations.PNameEn, dists)
        for destination, time in tups:
            key = (i.PNameEn, destination)
            if key not in trips:
                trips.add(key)
                trips.add((key[1], key[0]))
                f.write(",".join([key[0], key[1], str(time)]) + "\n")
                if key[0] != key[1]:
                    f.write(",".join([key[1], key[0], str(time)]) + "\n")


In [35]:
dist_iq_locations.to_csv('data/iq_dist_locations_test.csv')